In [ ]:
# Due to the file system setting of the Jupyter Notebook, navigate to the package to use infra
%cd /home/jupyter/quant

### Initial `CREATE TABLE` commands for tables database

In [ ]:
import pymysql
from yitian.datasource import *

# Set up cloud sql connections
password = ''
connection = pymysql.connect(host=PRIVATE_HOST,
                             user=USER,
                             password=password,
                             db=DATABASE)

#### Define the target table creation sql query -- using `command+/` to umcomment the chunk of codes

In [ ]:
# Create NASDAQ ticker table
sql = """
        CREATE TABLE IF NOT EXISTS nasdaq(
            symbol VARCHAR(10),
            name VARCHAR(75),
            marketcap VARCHAR(10),
            ipo_year INT,
            sector VARCHAR(25),
            industry VARCHAR(50),
            summary VARCHAR(100),
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL,
        PRIMARY KEY(symbol));
"""

# Create NYSE ticker table
sql = """
        CREATE TABLE IF NOT EXISTS nyse(
            symbol VARCHAR(10),
            name VARCHAR(75),
            marketcap VARCHAR(10),
            ipo_year INT,
            sector VARCHAR(25),
            industry VARCHAR(50),
            summary VARCHAR(100),
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL,
        PRIMARY KEY(symbol));
"""

# Create Stock Daily table
sql = """
        CREATE TABLE IF NOT EXISTS stock_daily(
            ticker VARCHAR(15),
            datetime TIMESTAMP NOT NULL,
            open FLOAT NOT NULL,
            high FLOAT NOT NULL,
            low FLOAT NOT NULL,
            close FLOAT NOT NULL,
            volume FLOAT DEFAULT 0.0 NOT NULL,
            year INT NOT NULL,
            month INT NOT NULL,
            day INT NOT NULL,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL,
        PRIMARY KEY(ticker, datetime)
        )
        PARTITION BY RANGE (year) (
            PARTITION p0 VALUES LESS THAN (1980),
            PARTITION p1 VALUES LESS THAN (1985),
            PARTITION p2 VALUES LESS THAN (1990),
            PARTITION p3 VALUES LESS THAN (1995),
            PARTITION p4 VALUES LESS THAN (2000),
            PARTITION p5 VALUES LESS THAN (2005),
            PARTITION p6 VALUES LESS THAN (2010),
            PARTITION p7 VALUES LESS THAN (2015),
            PARTITION p8 VALUES LESS THAN (2020),
            PARTITION p9 VALUES LESS THAN MAXVALUE
            );
        """

# Create Stock Hourly table
sql = """
        CREATE TABLE IF NOT EXISTS stock_hourly(
            ticker VARCHAR(15),
            datetime TIMESTAMP NOT NULL,
            open FLOAT NOT NULL,
            high FLOAT NOT NULL,
            low FLOAT NOT NULL,
            close FLOAT NOT NULL,
            volume FLOAT DEFAULT 0.0 NOT NULL,
            year INT NOT NULL,
            month INT NOT NULL,
            day INT NOT NULL,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL,
        PRIMARY KEY(ticker, datetime)
        )
        PARTITION BY RANGE (year) (
            PARTITION p0 VALUES LESS THAN (2015),
            PARTITION p1 VALUES LESS THAN (2016),
            PARTITION p2 VALUES LESS THAN (2017),
            PARTITION p3 VALUES LESS THAN (2018),
            PARTITION p4 VALUES LESS THAN (2019),
            PARTITION p5 VALUES LESS THAN (2020),
            PARTITION p6 VALUES LESS THAN (2021),
            PARTITION p7 VALUES LESS THAN (2022),
            PARTITION p8 VALUES LESS THAN (2023),
            PARTITION p9 VALUES LESS THAN (2024),
            PARTITION p10 VALUES LESS THAN MAXVALUE
            );
"""

# Create `Stock Actions (Dividends & Splits)` table
sql = """
        CREATE TABLE IF NOT EXISTS stock_actions(
            ticker VARCHAR(15),
            date DATE NOT NULL,
            dividends FLOAT NOT NULL,
            splits FLOAT NOT NULL,
        KEY(ticker, date));
"""

# Create `Major Holders` table
sql = """
        CREATE TABLE IF NOT EXISTS major_holders(
            ticker VARCHAR(15),
            updated_at DATE NOT NULL,
            insider_share_pct FLOAT NOT NULL,
            institution_share_pct FLOAT NOT NULL,
            institution_float_pct FLOAT NOT NULL,
            number_institution INT NOT NULL,
        KEY(ticker, updated_at));
"""

# Create `Institutional Holders` table
sql = """
        CREATE TABLE IF NOT EXISTS institutional_holders(
            ticker VARCHAR(15),
            holder VARCHAR(75) NOT NULL,
            shares BIGINT NOT NULL,
            date_reported DATE NOT NULL,
            out_pct FLOAT NOT NULL,
            value BIGINT NOT NULL,
        KEY(ticker, date_reported));
"""

# Create `Stock Recommendations` table
sql = """
        CREATE TABLE IF NOT EXISTS stock_recommendations(
            ticker VARCHAR(15),
            date DATE NOT NULL,
            firm VARCHAR(50) NOT NULL,
            to_grade VARCHAR(25),
            from_grade VARCHAR(25),
            action VARCHAR(25),
        KEY(ticker, date));
"""


In [ ]:
# Check the defined sql query
print(sql)

In [ ]:
# CREATE TABLES IF NOT EXIST
with connection.cursor() as cursor:
    
    cursor.execute(sql)
    
    # connection is not autocommit by default. So you must commit to save your changes.
    connection.commit()

cursor.close()